In [2]:
from bs4 import BeautifulSoup
import requests
from pyth.plugins.rtf15.reader import Rtf15Reader
from pyth.plugins.xhtml.writer import XHTMLWriter
import pandas as pd
import urllib2
from cStringIO import StringIO

import re


# For now just do all the rtf links
url = "https://www.gov.uk/government/statistics/prison-population-figures-2015"
r = requests.get(url)
soup = BeautifulSoup(r.content,"lxml")
s1 = set([(a.text, a["href"]) for a in soup.findAll("a") if ".rtf" in a["href"] and a.text != ""])

In [3]:
url = "https://www.gov.uk/government/statistics/prison-population-figures-2016"

import requests
r = requests.get(url)
soup = BeautifulSoup(r.content,"lxml")
s2 = set([(a.text, a["href"]) for a in soup.findAll("a") if ".rtf" in a["href"] and a.text != ""])

In [4]:
all_links = s1.union(s2)

In [5]:
def get_table(doc, source):
    final_data = []

    header = False
    for c in doc.content:
        full_p = c.content.__repr__().lower()
        if "capacity" in full_p and "use cna" in full_p:

            header = True
            continue

        if header:
            row= re.split(r"\t", c.content[0].content[0])
            if len(row) == 7 :
                final_data.append(row)
    df = pd.DataFrame(final_data, columns = ["prison_name","baseline_cna", "in_use_cna", "operational_capacity", "population", "perc_pop_to_used_cna", "perc_acc_available"])
    df.iloc[:,1:] = df.iloc[:,1:].replace("%", "", regex=True).replace(",", "", regex=True)

    for c in df.columns:
        df[c]= pd.to_numeric(df[c], errors='ignore')
    cols = [c for c in df.columns if "perc" in c]
    df.loc[:,cols] = df.loc[:,cols]/100
    
    all_text =  "\n".join([p.content[0].content[0] for p in doc.content])
    
    #Parse out the date here
    re_pattern = r"(\d{2}/\d{2}/\d{4})"
    

    date = re.search(re_pattern, all_text).group(1)   
    df["date"] = date
    df["source"] = source
    
    
    return df

In [7]:
dfs = []
for link in all_links:
    text = link[0]
    link = link[1]
    rtf_file = urllib2.urlopen("http://www.gov.uk" +link)
    rtf_file = StringIO(rtf_file.read())
    doc = Rtf15Reader.read(rtf_file)
    df = get_table(doc, "http://www.gov.uk" +link)
    dfs.append(df)
    

In [15]:
import urllib2
import os
for l in all_links:
    url = "https://www.gov.uk" + l[1]
    head, tail = os.path.split(url)
    my_file = urllib2.urlopen(url)
    
    with open("all_rtf/" + tail,'wb') as output:
      output.write(my_file.read())

In [10]:
final = pd.concat(dfs)
final["date"] = pd.to_datetime(final["date"],infer_datetime_format=True)

In [12]:
final.to_csv("all_pop_data_rtf.csv", index=False, encoding="utf-8")